In [1]:
#| default_exp Environments/HeterogeneousObservationsEnv

In [2]:
#| hide
# Imports for the nbdev development environment
from nbdev.showdoc import *
from fastcore.test import *

In [3]:
#| export
from fastcore.utils import *
import numpy as np

In [4]:
#| hide
%load_ext autoreload
%autoreload 2

In [5]:
#| export
class HeterogeneousObservationsEnv(object):
    def __init__(self, observation_opacity):

        self.observation_opacity = observation_opacity

        self.transitions = self.transition_tensor()
        self.final_states = np.array(self.generate_final_states())
        self.rewards = self.reward_tensor()
        self.observations = self.generate_observation_tensor()

        self.actions_set = self.actions()
        self.states_set = self.states() 
        self.observation_labels = self.generate_observation_labels()
        # R, T, O = self.rewards, self.transitions, self.observations

        self.n_agents = self.rewards.shape[0]
        self.n_states = self.transitions.shape[0]
        self.n_agent_actions = self.transitions.shape[1]

        # assert len(self.observation_opacity) == self.n_agents, 'Observation confidences need to be specified for all agents'

        # Checks
        # assert all(self.transitions.shape[1:-1] == self.n_agent_actions for _ in range(self.n_agents)), 'Inconsistent number of actions'
        assert all(dim == self.n_agent_actions for dim in self.rewards.shape[2:-1]), 'Inconsistent number of actions'
        assert len(self.actions_set) == self.n_agents and all(len(a) == self.n_agent_actions for a in self.actions_set), 'Inconsistent number of actions'
        assert self.transitions.shape[-1] == self.n_states and self.rewards.shape[-1] == self.n_states, 'Inconsistent number of states'
        assert self.rewards.shape[1] == self.n_states, 'Inconsistent number of states'
        assert len(self.final_states) == self.n_states, 'Inconsistent number of states'
        assert len(self.states_set) == self.n_states, 'Inconsistent number of states'
        assert np.allclose(self.transitions.sum(-1), 1), 'Transition model probabilities do not sum to 1'

        #     assert obs.shape[0] == self.n_agents, "Inconsistent number of agents"
        #     assert obs.shape[1] == self.n_states, "Inconsistent number of states"
        #     assert np.allclose(obs.sum(-1), 1), 'Observation model probabilities do not sum to 1'

In [9]:
#| export
@patch
def id(self:HeterogeneousObservationsEnv):
    """Returns id string of environment"""
    return f"{self.__class__.__name__}"

@patch
def __str__(self:HeterogeneousObservationsEnv): return self.id()

@patch
def __repr__(self:HeterogeneousObservationsEnv): return self.id()

@patch
def transition_tensor(self:HeterogeneousObservationsEnv):
    raise NotImplementedError
@patch
def reward_tensor(self:HeterogeneousObservationsEnv):
    raise NotImplementedError

@patch
def generate_observation_tensor(self:HeterogeneousObservationsEnv):
    """
    Generates the observation tensor for the environment. The tensor represents the probability distribution over possible
    observations given the actual state of the environment. The configuration of the tensor is influenced by the observation
    confidence of each agent, which is now a list of floats, with each float indicating the confidence level of the corresponding
    agent.
    """
    self.n_observations = self.n_states
    
    # if np.all(np.array(self.observation_opacity) > 0.5):
    #     self.n_observations = 1
    #     observations_iso = np.ones((self.n_agents, self.n_states, self.n_observations))
    # else:
    # Otherwise, set the number of observations equal to the number of states,
    # indicating a more detailed observation model.
    observations_iso = np.zeros((self.n_agents, self.n_states, self.n_observations))

    for agent_index in range(self.n_agents):
        print('for agent ', agent_index)
        # For each agent, configure the observation probabilities based on their individual observation confidence.
        # Adjust the observation tensor accordingly. The following assumes a binary state space for simplicity.
        # This logic may need to be extended for environments with more than two states.
        confidence = self.observation_opacity[agent_index]
        print('confidence is ,', confidence)
        print(self.n_agents, self.n_states, self.n_observations)
        # observations_iso = np.zeros((self.n_agents, self.n_states, self.n_observations))
        # print(f'observations_iso[{agent_index}, 0, 0] {1 - min(confidence, 0.5)}')
        # print(f'observations_iso[{agent_index}, 0, 1] {0 + min(confidence, 0.5)}')
        # print(f'observations_iso[{agent_index}, 1, 0] {0 + min(confidence, 0.5)}')
        # print(f'observations_iso[{agent_index}, 1, 1] {1 - min(confidence, 0.5)}')
        observations_iso[agent_index, 0, 0] = 1 - min(confidence, 0.5)
        observations_iso[agent_index, 0, 1] = 0 + min(confidence, 0.5)
        observations_iso[agent_index, 1, 0] = 0 + min(confidence, 0.5)
        observations_iso[agent_index, 1, 1] = 1 - min(confidence, 0.5)
    
    return observations_iso

# #| export
# @patch
# def ObservationTensor(self:UncertainSocialDilemma):

#         if np.all(self.noise > 0.5):
#             self.Q = 1
#             Oiso = np.ones((self.N, self.Z, self.Q))
            
#         else:
#             self.Q = self.Z
#             print((self.N, self.Z, self.Q))
#             print('==========================')
#             Oiso = np.zeros((self.N, self.Z, self.Q))

#             for i in range(self.N):
#                 Oiso[i,0,0] = 1 - min(self.noise[i], 0.5)
#                 Oiso[i,0,1] = 0 + min(self.noise[i], 0.5)
#                 Oiso[i,1,0] = 0 + min(self.noise[i], 0.5)
#                 Oiso[i,1,1] = 1 - min(self.noise[i], 0.5)
            
#         return Oiso



@patch
def generate_final_states(self:HeterogeneousObservationsEnv):
    """Default final states: no final states"""
    return np.zeros(self.n_states, dtype=int)

@patch
def actions(self:HeterogeneousObservationsEnv):
    """Default action set representations."""
    return [[str(a) for a in range(self.n_agent_actions)] for _ in range(self.n_agents)]

@patch
def states(self:HeterogeneousObservationsEnv):
    """Default state set representation."""
    return [str(s) for s in range(self.n_states)]

@patch
def generate_observation_labels(self:HeterogeneousObservationsEnv):
    """Creates observation labels."""
    return [[str(o) for o in range(self.n_observations)] for _ in range(self.n_agents)]

@patch
def step(self:HeterogeneousObservationsEnv, jA:Iterable) -> tuple:
    """Iterate the environment one step forward."""
    tps = self.transitions[tuple([self.state]+list(jA))].astype(float)
    next_state = np.random.choice(range(len(tps)), p=tps)
    rewards = self.rewards[tuple([slice(self.n_agents), self.state]+list(jA)+[next_state])]
    self.state = next_state
    obs = self.generate_stochastic_observations()
    done = self.state in np.where(self.final_states==1)[0]
    info = {'state': self.state}
    return obs, rewards.astype(float), done, info

@patch
def generate_stochastic_observations(self:HeterogeneousObservationsEnv) -> np.ndarray:
    """
    Produces a set of observations for each agent based on the current state, utilizing the defined observation tensors.
    Each tensor represents a different observation model, and this method generates observations according to the probability
    distributions specified in those tensors for the current state.
    
    Returns:
        A list of numpy arrays, where each array contains observations for all agents as determined by one of the observation tensors.
    """
    all_agents_observations = []  # Stores observations generated by each observation tensor.
    for observation_tensor in self.observations_list:
        current_state_observations = np.zeros(self.n_agents, dtype=int)  # Initializes the observation array for this tensor.
        for agent_index in range(self.n_agents):
            # Retrieves the probability distribution of observations for the current agent and state from the tensor.
            observation_probabilities = observation_tensor[agent_index, self.state]
            # Generates a random observation based on the probability distribution.
            chosen_observation = np.random.choice(range(len(observation_probabilities)), p=observation_probabilities)
            current_state_observations[agent_index] = chosen_observation
        all_agents_observations.append(current_state_observations)
    return all_agents_observations

In [7]:
#| hide
import nbdev; nbdev.nbdev_export()